In [21]:
#sample prediction
from fast_bert.prediction import BertClassificationPredictor

MODEL_PATH = '/home/sujitnashik/Downloads/model_out'
LABEL_PATH= '/home/sujitnashik/Downloads/nlp_data/'
predictor = BertClassificationPredictor(
				model_path=MODEL_PATH,
				label_path=LABEL_PATH,
				multi_label=False,
				model_type='bert',
				do_lower_case=False,
				device=None)




texts = [
	"this is the first text",
	"this is complex text"
	]

multiple_predictions = predictor.predict_batch(texts)
print(multiple_predictions)

[[('neutral', 0.8142256140708923), ('joy', 0.16540467739105225), ('surprise', 0.01722590997815132), ('anger', 0.002016712911427021), ('sadness', 0.000653256953228265), ('fear', 0.00027571909595280886), ('disgust', 0.0001981032983167097)], [('neutral', 0.9364937543869019), ('joy', 0.046546194702386856), ('surprise', 0.012686217203736305), ('anger', 0.0023554586805403233), ('sadness', 0.0007997110951691866), ('fear', 0.0007758369320072234), ('disgust', 0.0003428677446208894)]]


In [15]:
#prediction final
from collections import defaultdict
import pandas as pd
from fast_bert.prediction import BertClassificationPredictor

MODEL_PATH = '/home/sujitnashik/Downloads/model_out'
LABEL_PATH = '/home/sujitnashik/Downloads/nlp_data/'
labels_list = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']

predictor = BertClassificationPredictor(
    model_path=MODEL_PATH,
    label_path=LABEL_PATH,
    multi_label=False,
    model_type='bert',
    do_lower_case=False,
    device=None)

# Define a mapping of original labels to positive, negative, and neutral
emotion_mapping = {
    'anger': 'negative', 'disgust': 'negative', 'fear': 'negative',
    'joy': 'positive', 'sadness': 'negative', 'surprise': 'positive',
    'neutral': 'neutral'
}# Define the data as a string
data = """
Original Dialogue:
A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: I hate you?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we’ve discussed it many times. I think he’s ready now.
B: That’s good. Raising a dog is a tough issue. Like having a baby ;-) 
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))

Reference Summary:
A will go to the animal shelter tomorrow to get a puppy for her son. They already visited the shelter last Monday and the son chose the puppy.

Model-generated Summary:
[{'summary_text': 'A wants to get a puppy for her son. A took him to the animal shelter last Monday and he wanted to drag him away.'}]

"""

# Split the data into parts and then split the dialogue into lines
parts = data.split('\n\n')

original_dialogue_lines = parts[0].split('\n')[1:]  # Skip the title and split into lines

emotion_counts = defaultdict(lambda: {'positive': 0, 'negative': 0, 'neutral': 0})

# Process each line of dialogue
for line in original_dialogue_lines:
    if ": " in line:
        speaker, text = line.split(": ", 1)
        predictions = predictor.predict(text)
        top_label = predictions[0][0] if predictions else 'neutral'
        mapped_label = emotion_mapping.get(top_label, 'neutral')
        print(f"{speaker}: Text: '{text}', Prediction: '{top_label}', Mapped Emotion: '{mapped_label}'")
        emotion_counts[speaker][mapped_label] += 1

# Process the summary
summary_text = parts[1].split('\n')[1]
summary_prediction = predictor.predict(summary_text)
summary_top_label = summary_prediction[0][0] if summary_prediction else 'neutral'
summary_mapped_label = emotion_mapping.get(summary_top_label, 'neutral')
print(f"\nSummary: '{summary_text}', Summary Emotion: {summary_mapped_label}")
emotion_df = pd.DataFrame(emotion_counts).fillna(0).astype(int).transpose()

model_generated_summary = eval(parts[2].split('\n')[1])[0]["summary_text"]
model_summary_prediction = predictor.predict(model_generated_summary)
model_summary_top_label = model_summary_prediction[0][0] if model_summary_prediction else 'neutral'
model_summary_mapped_label = emotion_mapping.get(model_summary_top_label, 'neutral')
print(f"\nModel-generated Summary: '{model_generated_summary}', Summary Emotion: {model_summary_mapped_label}")

for user, emotions in emotion_counts.items():
    net_emotion = emotions['positive'] - emotions['negative']
    emotion_counts[user]['net_emotion'] = net_emotion


emotion_df = pd.DataFrame(emotion_counts).fillna(0).astype(int).transpose()


emotion_df['Overall Sentiment'] = emotion_df.apply(lambda row: 'Positive' if row['net_emotion'] > 0 else ('Negative' if row['net_emotion'] < 0 else 'Neutral'), axis=1)

print(emotion_df)



A: Text: 'Hi Tom, are you busy tomorrow’s afternoon?', Prediction: 'surprise', Mapped Emotion: 'positive'
B: Text: 'I’m pretty sure I am. What’s up?', Prediction: 'joy', Mapped Emotion: 'positive'
A: Text: 'Can you go with me to the animal shelter?.', Prediction: 'surprise', Mapped Emotion: 'positive'
B: Text: 'I hate you?', Prediction: 'anger', Mapped Emotion: 'negative'
A: Text: 'I want to get a puppy for my son.', Prediction: 'joy', Mapped Emotion: 'positive'
B: Text: 'That will make him so happy.', Prediction: 'joy', Mapped Emotion: 'positive'
A: Text: 'Yeah, we’ve discussed it many times. I think he’s ready now.', Prediction: 'joy', Mapped Emotion: 'positive'
B: Text: 'That’s good. Raising a dog is a tough issue. Like having a baby ;-) ', Prediction: 'joy', Mapped Emotion: 'positive'
A: Text: 'I'll get him one of those little dogs.', Prediction: 'neutral', Mapped Emotion: 'neutral'
B: Text: 'One that won't grow up too big;-)', Prediction: 'neutral', Mapped Emotion: 'neutral'
A: Te